In [1]:
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
inpath = '/Users/yanmemgxing/Nutstore Files/scientific group/'

In [10]:
group_info = pd.read_csv(f'{inpath}survival/groupinfo(G1 mentor).csv')

In [11]:
group_info.head(5)

,work_id,author_position,author_id,institution_id,publication_year,first_pub_year,scientific_age,group_size,mentor_id,current_year,ratio_institution,current_scientific_age,mentor_first_pub_year,mentor_scientific_age,mentor_current_scientific_age,end_year,career_len,average_group_size
0,W1987398976,first,A1973568665,I126307644,1989,1988,1,2,A1095119240,1989,1.0,1,1985,4,4,1992.0,33.0,10.0
1,W2009362099,first,A1973568665,I126307644,1989,1988,1,2,A1095119240,1989,1.0,1,1985,4,4,1992.0,33.0,10.0
2,W2128354159,first,A2350033295,I126307644,1989,1985,4,2,A1095119240,1989,1.0,4,1985,4,4,NaN,NaN,NaN
3,W1987398976,first,A1973568665,I126307644,1989,1988,1,2,A1095119240,1990,1.0,2,1985,4,5,1992.0,33.0,10.0
4,W2009362099,first,A1973568665,I126307644,1989,1988,1,2,A1095119240,1990,1.0,2,1985,4,5,1992.0,33.0,10.0


##### filter condition

In [15]:
group_info = group_info[(group_info.current_scientific_age<6)&(group_info.mentor_scientific_age>10)]

In [16]:
au_i = 'A1973568665'
group_info[group_info.author_id==au_i]

,work_id,author_position,author_id,institution_id,publication_year,first_pub_year,scientific_age,group_size,mentor_id,current_year,ratio_institution,current_scientific_age,mentor_first_pub_year,mentor_scientific_age,mentor_current_scientific_age,end_year,career_len,average_group_size
2122583,W1981486388,middle,A1973568665,I135310074,1992,1988,4,8,A875816979,1992,0.486486,4,1969,23,23,1992.0,33.0,10.0
2122593,W1981486388,middle,A1973568665,I135310074,1992,1988,4,11,A875816979,1993,0.547619,5,1969,23,24,1992.0,33.0,10.0


##### the mentee's work

In [17]:
all_mentee = group_info.author_id.drop_duplicates().tolist()
print('num_of_mentee: ',len(all_mentee))

num_of_mentee:  204647


In [18]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}?charset=utf8mb4"
                       .format(user="openalex_ym",
                               pw="openalex.ym",
                               host="172.18.18.167",
                               port=3306,
                               db="openalex2022"),
                       )

In [20]:
mentee_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = all_mentee[start_point:end_point]
        mentee_groups.append(temp)
    except:
        temp = all_mentee[start_point:]
        mentee_groups.append(temp)
        
    start_point = end_point
    if end_point>len(all_mentee):
        break
print(len(mentee_groups))

41


In [21]:
l = len(mentee_groups)
print(l)
for i in range(l):
    temp =  mentee_groups[i]
    sql_query = """select works_authorships.work_id, works_authorships.author_id

                    from works_authorships 

                    where works_authorships.author_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    if i%10==0:
        print(i)

Data = tp

41
0
10
20
30
40


In [22]:
Data = Data.rename(columns={'id':'work_id'})

In [24]:
# Data.to_csv('G1_mentees_works.csv',index=0)

##### find work's performance

In [25]:
alex_mentee_work = Data

In [26]:
alex_mentee_work.head(5)

,work_id,author_id
0,W2183897491,A100345908
1,W2081058593,A100345908
2,W2025851120,A100345908
3,W1983338691,A100345908
4,W2107552544,A100345908


In [27]:
works = alex_mentee_work.work_id.drop_duplicates().tolist()

In [28]:
works_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = works[start_point:end_point]
        works_groups.append(temp)
    except:
        temp = works[start_point:]
        works_groups.append(temp)
        
    start_point = end_point
    if end_point>len(works):
        break
l = len(works_groups)
print(l)

740


In [29]:
for i in range(l):
    temp =  works_groups[i]
    sql_query = """select works_puby_cits_team_concepts_innovation.id, works_puby_cits_team_concepts_innovation.cits,works_puby_cits_team_concepts_innovation.cits5

                    from works_puby_cits_team_concepts_innovation 

                    where works_puby_cits_team_concepts_innovation.id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    if i%10==0:
        print(i)
Data = tp

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730


In [30]:
work_cits = Data
work_cits = work_cits.rename(columns={'id':'work_id'})

In [31]:
alex_mentee_work_cits = alex_mentee_work.merge(work_cits,on='work_id',how='left')

In [32]:
total_cits = alex_mentee_work_cits.groupby('author_id')['cits'].agg('sum').reset_index(name='total_cits')
total_cits5 = alex_mentee_work_cits.groupby('author_id')['cits5'].agg('sum').reset_index(name='total_cits5')

In [33]:
cits = total_cits.merge(total_cits5,on='author_id',how='left')

In [35]:
paper_num = alex_mentee_work.groupby('author_id')['work_id'].agg('count').reset_index(name='paper_num')
cits = cits.merge(paper_num,how='left',on='author_id')

In [36]:
cits.head(5)

,author_id,total_cits,total_cits5,paper_num
0,A1000511781,88.0,45.0,6
1,A100052661,0.0,0.0,6
2,A100174395,881.0,611.0,58
3,A10018131,846.0,398.0,30
4,A100238711,277.0,159.0,11


In [37]:
group_info = group_info.merge(cits,on='author_id',how='left')

In [40]:
group_info['yearly_cits'] = group_info['total_cits']/(group_info['career_len']+1)
group_info['yearly_cits5'] = group_info['total_cits5']/(group_info['career_len']+1)

In [42]:
# group_info.to_csv('groupinfo(filted G1 mentor).csv',index=0)

In [43]:
group_info.head(5)

,work_id,author_position,author_id,institution_id,publication_year,first_pub_year,scientific_age,group_size,mentor_id,current_year,...,mentor_scientific_age,mentor_current_scientific_age,end_year,career_len,average_group_size,total_cits,total_cits5,paper_num,yearly_cits,yearly_cits5
0,W2138316501,first,A2066475195,I126307644,1996,1994,2,6,A1095119240,1996,...,11,11,1996.0,2.0,5.0,299.0,79.0,4,99.666667,26.333333
1,W1985050512,first,A2662327865,I126307644,1996,1996,0,6,A1095119240,1996,...,11,11,1996.0,0.0,6.0,808.0,155.0,4,808.000000,155.000000
2,W2082813816,first,A2689291091,I126307644,1996,1996,0,6,A1095119240,1996,...,11,11,1996.0,0.0,6.0,16.0,4.0,1,16.000000,4.000000
3,W2138316501,first,A2066475195,I126307644,1996,1994,2,5,A1095119240,1997,...,11,12,1996.0,2.0,5.0,299.0,79.0,4,99.666667,26.333333
4,W1985050512,first,A2662327865,I126307644,1996,1996,0,5,A1095119240,1997,...,11,12,1996.0,0.0,6.0,808.0,155.0,4,808.000000,155.000000


In [44]:
def average_performance(data,current_year,drop_criteria):
    group_info_2000 = data[['author_id','average_group_size','career_len','paper_num','yearly_cits','yearly_cits5',
                            'total_cits','total_cits5']][data.end_year==current_year].drop_duplicates()  
    
    num_of_mentee = len(group_info_2000)
    # print(len(group_info_2000))
    Q75 = np.quantile(group_info_2000['average_group_size'],0.75)
    Q50 = np.quantile(group_info_2000['average_group_size'],0.50)
    Q25 = np.quantile(group_info_2000['average_group_size'],0.25)
    Q10 = np.quantile(group_info_2000['average_group_size'],0.10)
    
    data_temp = []
    # BigGroup
    G75 = group_info_2000[group_info_2000.average_group_size>=Q75]
    performance_mean = G75[['career_len', 'paper_num', 'yearly_cits', 'yearly_cits5',
           'total_cits', 'total_cits5']].mean().tolist()
    
    performance_mean.append(current_year)
    performance_mean.append(drop_criteria)
    performance_mean.append('BigGroup')
    performance_mean.append('AllMentee')
    
    
    G75_surviver = G75[G75.career_len>drop_criteria]
    performance_mean_surviver = G75_surviver[['career_len', 'paper_num', 'yearly_cits', 'yearly_cits5','total_cits', 'total_cits5']].mean().tolist()

    performance_mean_surviver.append(current_year)
    performance_mean_surviver.append(drop_criteria)
    performance_mean_surviver.append('BigGroup')
    performance_mean_surviver.append('SurvivalMentee')
    
    data_temp.append(performance_mean)
    data_temp.append(performance_mean_surviver)
    
    #SmallGroup
    G25 = group_info_2000[group_info_2000.average_group_size<=Q25]
    performance_mean = G25[['career_len', 'paper_num', 'yearly_cits', 'yearly_cits5',
           'total_cits', 'total_cits5']].mean().tolist()
    performance_mean.append(current_year)
    performance_mean.append(drop_criteria)
    performance_mean.append('SmallGroup')
    performance_mean.append('AllMentee')
    G25_surviver = G25[G25.career_len>drop_criteria]
    performance_mean_surviver = G25_surviver[['career_len', 'paper_num', 'yearly_cits', 'yearly_cits5','total_cits', 'total_cits5']].mean().tolist()
    performance_mean_surviver.append(current_year)
    performance_mean_surviver.append(drop_criteria)
    performance_mean_surviver.append('SmallGroup')
    performance_mean_surviver.append('SurvivalMentee')
    
    data_temp.append(performance_mean)
    data_temp.append(performance_mean_surviver)
    
    return data_temp

In [46]:
average_performance(group_info,2005,10)

[[7.4941383352872215,
  30.19284876905041,
  119.93857612940042,
  56.4825828491837,
  1356.236225087925,
  736.0762016412662,
  2005,
  10,
  'BigGroup',
  'AllMentee'],
 [17.063716814159292,
  77.66902654867256,
  170.36148938842962,
  98.86377698376998,
  3268.895575221239,
  1908.4920353982302,
  2005,
  10,
  'BigGroup',
  'SurvivalMentee'],
 [8.89592050209205,
  38.2081589958159,
  84.80052835465607,
  43.65018221958305,
  1166.8436192468619,
  670.9262552301255,
  2005,
  10,
  'SmallGroup',
  'AllMentee'],
 [17.423707440100884,
  82.63682219419924,
  127.09379896673991,
  75.28515704489145,
  2492.6960907944513,
  1480.965952080706,
  2005,
  10,
  'SmallGroup',
  'SurvivalMentee']]

In [47]:
#constant variable
data = group_info

Data = []
years = list(range(1980,2015+1))
Drop_Criteria = [5,10,15,20]

for drop_criteria in Drop_Criteria:
    for current_year in years:
        print(drop_criteria,current_year)
        temp = average_performance(data,current_year,drop_criteria)
        Data.extend(temp)
        # print('survival rate in Big group :', '{:.2%}'.format(big))
        # print('survival rate in Small group :', '{:.2%}'.format(small))


5 1980
5 1981
5 1982
5 1983
5 1984
5 1985
5 1986
5 1987
5 1988
5 1989
5 1990
5 1991
5 1992
5 1993
5 1994
5 1995
5 1996
5 1997
5 1998
5 1999
5 2000
5 2001
5 2002
5 2003
5 2004
5 2005
5 2006
5 2007
5 2008
5 2009
5 2010
5 2011
5 2012
5 2013
5 2014
5 2015
10 1980
10 1981
10 1982
10 1983
10 1984
10 1985
10 1986
10 1987
10 1988
10 1989
10 1990
10 1991
10 1992
10 1993
10 1994
10 1995
10 1996
10 1997
10 1998
10 1999
10 2000
10 2001
10 2002
10 2003
10 2004
10 2005
10 2006
10 2007
10 2008
10 2009
10 2010
10 2011
10 2012
10 2013
10 2014
10 2015
15 1980
15 1981
15 1982
15 1983
15 1984
15 1985
15 1986
15 1987
15 1988
15 1989
15 1990
15 1991
15 1992
15 1993
15 1994
15 1995
15 1996
15 1997
15 1998
15 1999
15 2000
15 2001
15 2002
15 2003
15 2004
15 2005
15 2006
15 2007
15 2008
15 2009
15 2010
15 2011
15 2012
15 2013
15 2014
15 2015
20 1980
20 1981
20 1982
20 1983
20 1984
20 1985
20 1986
20 1987
20 1988
20 1989
20 1990
20 1991
20 1992
20 1993
20 1994
20 1995
20 1996
20 1997
20 1998
20 1999
20 2000
20 2

In [48]:
Data = pd.DataFrame(Data,columns=['career_len', 'paper_num', 'yearly_cits', 'yearly_cits5','total_cits', 'total_cits5','current_year','drop_criteria','Big_or_Small','All_or_Survival'])

In [49]:
Data.head(5)

,career_len,paper_num,yearly_cits,yearly_cits5,total_cits,total_cits5,current_year,drop_criteria,Big_or_Small,All_or_Survival
0,9.894737,29.562753,81.641923,23.684232,1376.315789,513.214575,1980,5,BigGroup,AllMentee
1,28.818182,87.701299,111.787509,42.055047,4054.896104,1561.974026,1980,5,BigGroup,SurvivalMentee
2,13.250000,53.921642,81.834509,24.811356,2105.638060,792.798507,1980,5,SmallGroup,AllMentee
3,31.358491,131.924528,123.877816,46.026510,5084.915094,1961.141509,1980,5,SmallGroup,SurvivalMentee
4,9.834483,45.482759,85.300521,30.545808,1928.141379,754.555172,1981,5,BigGroup,AllMentee


In [54]:
drop_criteria = 10
start = 1990
end = start
Data[(Data.current_year>=start)&(Data.current_year<=end)&(Data.drop_criteria==drop_criteria)
     &(Data.All_or_Survival=='AllMentee')]

,career_len,paper_num,yearly_cits,yearly_cits5,total_cits,total_cits5,current_year,drop_criteria,Big_or_Small,All_or_Survival
184,7.822511,36.994228,93.869557,35.074113,1521.272727,676.326118,1990,10,BigGroup,AllMentee
186,12.668374,53.185659,84.562172,32.334174,1980.848912,901.891165,1990,10,SmallGroup,AllMentee


In [55]:
drop_criteria = 10
start = start
end = start
Data[(Data.current_year>=start)&(Data.current_year<=end)&(Data.drop_criteria==drop_criteria)
     &(Data.All_or_Survival=='SurvivalMentee')]

,career_len,paper_num,yearly_cits,yearly_cits5,total_cits,total_cits5,current_year,drop_criteria,Big_or_Small,All_or_Survival
185,28.720779,151.181818,183.574893,83.512073,6139.480519,2820.837662,1990,10,BigGroup,SurvivalMentee
187,29.125000,130.256579,142.868941,65.973042,4859.279605,2261.434211,1990,10,SmallGroup,SurvivalMentee
